# (iv) Reduced Input Model Development

## Model Development and Comparison

| Trial No | Nodes in Hidden Layer | Features | MSE |   |   |   | R² |   |   |   |
|----------|----------------------|----------|-----|---|---|---|-------|---|---|---|
|          |                      |          | Trn | Val | Test | All | Trn | Val | Test | All |
| A        | 5                    | Selected | 1.0794 | 0.1992 | 0.2340 | 0.7302 | 0.9861 | 0.9971 | 0.9950 | 0.9895 |
| B        | 10                   | Selected | 0.9979 | 0.2352 | 1.1106 | 0.8664 | 0.9871 | 0.9966 | 0.9761 | 0.9876 |
| C        | 20                   | Selected | 0.6627 | 0.1270 | 0.3249 | 0.4859 | 0.9915 | 0.9981 | 0.9930 | 0.9930 |

## Best Network Analysis

### Comparison with Full-Input Models

1. **Best Reduced Input Model:**
   - 5 neurons in hidden layer
   - Test R² = 0.9950
   - Test MSE = 0.2340

2. **Best Full-Input Model:**
   - 20 neurons in hidden layer
   - Test R² = 0.9922
   - Test MSE = 0.3647

## Key Insights

1. **Model Simplification:**
   - The reduced input model achieves superior performance with fewer neurons (5 vs 20)
   - Test error is lower (MSE 0.2340 vs 0.3647)
   - Prediction accuracy is higher (R² 0.9950 vs 0.9922)

2. **Efficiency Gains:**
   - Reduced measurement requirements (9 inputs vs 14)
   - Simpler network architecture (5 neurons vs 20)
   - Lower computational complexity

3. **Performance Consistency:**
   - Both models show excellent generalization
   - Reduced input model shows more consistent performance across training, validation, and test sets

## Possibility of Effective Smaller Model

The results strongly support the viability of a smaller, more efficient model:

1. **Practical Benefits:**
   - Reduces data collection costs by requiring fewer measurements
   - Simplifies the measurement process
   - Maintains high accuracy despite reduced inputs

2. **Model Advantages:**
   - More parsimonious architecture
   - Potentially faster training and inference
   - Reduced risk of overfitting

3. **Clinical Implications:**
   - More efficient body fat assessment process
   - Potential for wider adoption due to simplified requirements
   - Maintains high accuracy for reliable health assessment

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
import tensorflow as tf

# Based on best results from part (ii)
def create_and_train_model(X_train, X_val, X_test, y_train, y_val, y_test, 
                          hidden_neurons, learning_rate=0.1, batch_size=32, seed=123):
    # Set random seed for reproducibility
    tf.random.set_seed(seed)
    np.random.seed(seed)
    
    # Create model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(hidden_neurons, activation='sigmoid'))
    model.add(Dense(1, activation='linear'))
    
    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    
    # Early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=100,
        restore_best_weights=True
    )
    
    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20000,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Evaluate model
    X_all = np.vstack([X_train, X_val, X_test])
    y_all = np.concatenate([y_train, y_val, y_test])
    
    # Make predictions
    y_pred_train = model.predict(X_train, verbose=0)
    y_pred_val = model.predict(X_val, verbose=0)
    y_pred_test = model.predict(X_test, verbose=0)
    y_pred_all = model.predict(X_all, verbose=0)
    
    # Calculate MSE
    mse_train = np.mean((y_train - y_pred_train.flatten()) ** 2)
    mse_val = np.mean((y_val - y_pred_val.flatten()) ** 2)
    mse_test = np.mean((y_test - y_pred_test.flatten()) ** 2)
    mse_all = np.mean((y_all - y_pred_all.flatten()) ** 2)
    
    # Calculate R²
    r2_train = r2_score(y_train, y_pred_train)
    r2_val = r2_score(y_val, y_pred_val)
    r2_test = r2_score(y_test, y_pred_test)
    r2_all = r2_score(y_all, y_pred_all)
    
    return {
        'mse': {'train': mse_train, 'val': mse_val, 'test': mse_test, 'all': mse_all},
        'r2': {'train': r2_train, 'val': r2_val, 'test': r2_test, 'all': r2_all}
    }

def prepare_data(selected_features=None):
    # Load the data
    df = pd.read_csv('Body_Fat.csv')
    
    if selected_features is None:
        X = df.drop('BodyFat', axis=1)
    else:
        X = df[selected_features]
    
    y = df['BodyFat']
    
    # Scale the features
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Split the data
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.25, random_state=42
    )
    
    return X_train, X_val, X_test, y_train, y_val, y_test

def run_experiments():
    # Strong correlations identified in part (iii)
    selected_features = ['Density', 'Abdomen', 'Chest', 'Hip', 'Weight', 
                         'Thigh', 'Knee', 'Biceps', 'Neck']
    
    print("Model 1: Using all features")
    X_train_all, X_val_all, X_test_all, y_train, y_val, y_test = prepare_data()
    
    print("Model 2: Using selected features")
    X_train_selected, X_val_selected, X_test_selected, _, _, _ = prepare_data(selected_features)
    
    results = []
    for neurons in [5, 10, 20]:
        # Train model with all features
        result_all = create_and_train_model(
            X_train_all, X_val_all, X_test_all, y_train, y_val, y_test, neurons
        )
        results.append({
            'neurons': neurons,
            'features': 'all',
            **result_all
        })
        
        # Train model with selected features
        result_selected = create_and_train_model(
            X_train_selected, X_val_selected, X_test_selected, y_train, y_val, y_test, neurons
        )
        results.append({
            'neurons': neurons,
            'features': 'selected',
            **result_selected
        })
        
        # Print results
        print(f"\nNeurons: {neurons}")
        print("All Features:")
        print(f"MSE - Train: {result_all['mse']['train']:.4f}, Val: {result_all['mse']['val']:.4f}, "
              f"Test: {result_all['mse']['test']:.4f}, All: {result_all['mse']['all']:.4f}")
        print(f"R² - Train: {result_all['r2']['train']:.4f}, Val: {result_all['r2']['val']:.4f}, "
              f"Test: {result_all['r2']['test']:.4f}, All: {result_all['r2']['all']:.4f}")
        
        print("\nSelected Features:")
        print(f"MSE - Train: {result_selected['mse']['train']:.4f}, Val: {result_selected['mse']['val']:.4f}, "
              f"Test: {result_selected['mse']['test']:.4f}, All: {result_selected['mse']['all']:.4f}")
        print(f"R² - Train: {result_selected['r2']['train']:.4f}, Val: {result_selected['r2']['val']:.4f}, "
              f"Test: {result_selected['r2']['test']:.4f}, All: {result_selected['r2']['all']:.4f}")
        print("-" * 80)
    
    # Find best models
    best_all = max([r for r in results if r['features'] == 'all'], 
                   key=lambda x: x['r2']['test'])
    best_selected = max([r for r in results if r['features'] == 'selected'], 
                        key=lambda x: x['r2']['test'])
    
    print("\nBest Model with All Features:")
    print(f"Neurons: {best_all['neurons']}")
    print(f"Test R²: {best_all['r2']['test']:.4f}")
    print(f"Test MSE: {best_all['mse']['test']:.4f}")
    
    print("\nBest Model with Selected Features:")
    print(f"Neurons: {best_selected['neurons']}")
    print(f"Test R²: {best_selected['r2']['test']:.4f}")
    print(f"Test MSE: {best_selected['mse']['test']:.4f}")
    
    return results, best_all, best_selected

if __name__ == "__main__":
    results, best_all, best_selected = run_experiments()

Model 1: Using all features
Model 2: Using selected features

Neurons: 5
All Features:
MSE - Train: 1.7193, Val: 0.5089, Test: 0.5141, All: 1.2304
R² - Train: 0.9778, Val: 0.9926, Test: 0.9889, All: 0.9824

Selected Features:
MSE - Train: 1.0794, Val: 0.1992, Test: 0.2340, All: 0.7302
R² - Train: 0.9861, Val: 0.9971, Test: 0.9950, All: 0.9895
--------------------------------------------------------------------------------

Neurons: 10
All Features:
MSE - Train: 0.7903, Val: 0.3538, Test: 0.5872, All: 0.6609
R² - Train: 0.9898, Val: 0.9948, Test: 0.9874, All: 0.9905

Selected Features:
MSE - Train: 0.9979, Val: 0.2352, Test: 1.1106, All: 0.8664
R² - Train: 0.9871, Val: 0.9966, Test: 0.9761, All: 0.9876
--------------------------------------------------------------------------------

Neurons: 20
All Features:
MSE - Train: 0.8498, Val: 0.3190, Test: 0.3647, All: 0.6442
R² - Train: 0.9890, Val: 0.9953, Test: 0.9922, All: 0.9908

Selected Features:
MSE - Train: 0.6627, Val: 0.1270, Test: 0.